In [ ]:
import SALib as sa
from SALib import ProblemSpec
from SALib.analyze import sobol
import numpy as np
import sys
sys.path.append("../scripts")
from model import DuneModel
from experiment_utils import *
sys.path.remove("../scripts")

In [ ]:
def salib_wrapper(target: str, **kwargs):
    """ Wrapper for SALib to use the DuneModel, since there can be multiple outputs, this returns a new function specifying the target output """
    def out_fun(inputs):
        result = np.empty(len(inputs))
        for i, inp in enumerate(inputs):
            m = DuneModel(width=inp[0].astype(int),
                height=inp[1].astype(int),
                n_tribes=inp[2].astype(int),
                n_agents=inp[3].astype(int),
                n_heaps=inp[4].astype(int),
                vision_radius=inp[5].astype(int),
                step_count=inp[6].astype(int),
                alpha=inp[7],
                trade_percentage=inp[8],
                **kwargs)
            out = m.run_model()
            result[i] = out[target].to_numpy()[-1]
        return result

    return out_fun

In [ ]:
# Define the problem specification dictionary
problem = ProblemSpec({
    'names': [
    "width",
    "height",
    "n_tribes",
    "n_agents",
    "n_heaps",
    "vision_radius",
    "step_count",
    "alpha",
    "trade_percentage",
],
    'bounds': [
    (10, 200),  # width
    (10, 200),  # height
    (1, 10),    # n_tribes
    (10, 100),  # n_agents
    (1, 10),    # n_heaps
    (1, 10),    # vision_radius
    (50, 500),   # step_count
    (0.0, 1.0), # alpha
    (0.0, 1.0), # trade_percentage
    ]
})



In [ ]:
random_spice_map_picker = lambda *args: np.random.choice([gen_spice_map])( *args)
random_river_picker = lambda *args: np.random.choice([gen_river_random, gen_river_line, no_river])( *args)
random_location_picker = lambda *args: np.random.choice([random_locations, split_tribe_locations])( *args)

In [ ]:

kwargs = {
    "experiment_name": "test",
    "spice_generator": pice_map_picker,
    "river_generator": river_picker,
    "location_generator": location_picker,
    "spice_kwargs": {"cov_range": (3, 9), "total_spice": 1000}
}

In [ ]:
sensitivity_target_nomads = salib_wrapper('Nomads', **kwargs)
sa.ProblemSpec.evaluate_parallel
# Generate samples using Sobol sequence
samples = problem.sample_sobol(1024)

# Evaluate the model
results = samples.evaluate_parallel(sensitivity_target_nomads)


In [ ]:

Si = results.analyze_sobol()
print(Si)

Si.plot()